In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
rsi_data = pd.read_csv("/Users/fuadhassan/Desktop/sketch/OwlHack_2024_Finance_Challenge-main/output/RandomStrategy_data.csv")
macd_data = pd.read_csv("/Users/fuadhassan/Desktop/sketch/OwlHack_2024_Finance_Challenge-main/output/MACDStrategy_data.csv")

In [3]:
# Merge the RSI and MACD data on the 'Date' column
merged_data = pd.merge(rsi_data, macd_data, on='Date', how="inner", suffixes=('', '_macd'))

# Display the first few rows to confirm the merge
merged_data.head()

,Date,SPY_Open,SPY_High,SPY_Low,SPY_Close,SPY_Volume,SPY_Dividends,SPY_Stock Splits,SPY_Capital Gains,SPY_Ticker,...,SPY_SMA_20_macd,SPY_SMA_50_macd,SPY_SMA_200_macd,SPY_daily_returns_macd,SPY_weekly_returns_macd,SPY_monthly_returns_macd,SPY_RSI_macd,SPY_RSI2_macd,SPY_MACD_macd,SPY_MACD_Signal_macd
0,2016-01-04,173.584324,174.051851,171.939296,174.043198,222353500,0.0,0.0,0.0,SPY,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,NaN,NaN,NaN
1,2016-01-05,174.372212,174.805112,173.203389,174.337585,110845800,0.0,0.0,0.0,SPY,...,0.0,0.0,0.0,0.001691,0.0,0.0,0.0,NaN,NaN,NaN
2,2016-01-06,171.722865,173.212042,171.082181,172.138458,152112600,0.0,0.0,0.0,SPY,...,0.0,0.0,0.0,-0.012614,0.0,0.0,0.0,NaN,NaN,NaN
3,2016-01-07,169.116814,170.943654,167.610317,168.008591,213436100,0.0,0.0,0.0,SPY,...,0.0,0.0,0.0,-0.023992,0.0,0.0,0.0,NaN,NaN,NaN
4,2016-01-08,168.995584,169.567016,165.870044,166.164413,209817200,0.0,0.0,0.0,SPY,...,0.0,0.0,0.0,-0.010977,0.0,0.0,0.0,NaN,NaN,NaN


In [4]:
# Calculate the correlation matrix to observe relationships between indicators and SPY_daily_returns
correlation_matrix = merged_data[['SPY_daily_returns', 'SPY_RSI', 'SPY_RSI2', 'SPY_MACD', 'SPY_MACD_Signal']].corr()

correlation_matrix

,SPY_daily_returns,SPY_RSI,SPY_RSI2,SPY_MACD,SPY_MACD_Signal
SPY_daily_returns,1.000000,0.294338,0.312022,0.017259,-0.038678
SPY_RSI,0.294338,1.000000,1.000000,0.791223,0.640758
SPY_RSI2,0.312022,1.000000,1.000000,0.791223,0.640758
SPY_MACD,0.017259,0.791223,0.791223,1.000000,0.937367
SPY_MACD_Signal,-0.038678,0.640758,0.640758,0.937367,1.000000


The correlation matrix reveals:

* SPY_daily_returns has a modest positive correlation with both SPY_RSI (0.29) and SPY_RSI2 (0.31), indicating some influence of RSI on daily returns.
* SPY_MACD has a low correlation with SPY_daily_returns (0.02), while SPY_MACD_Signal has a slightly negative correlation (-0.04).

In [5]:
# Define buy and sell signals based on RSI and MACD thresholds
def generate_signals(data):
    """
    Generate buy and sell signals based on RSI and MACD indicators.
    Buy Signal: RSI < 30 (oversold) and MACD > MACD_Signal (uptrend)
    Sell Signal: RSI > 70 (overbought) and MACD < MACD_Signal (downtrend)
    """
    data['Signal'] = 0  # 0 for hold, 1 for buy, -1 for sell
    data.loc[(data['SPY_RSI'] < 30) & (data['SPY_MACD'] > data['SPY_MACD_Signal']), 'Signal'] = 1  # Buy signal
    data.loc[(data['SPY_RSI'] > 70) & (data['SPY_MACD'] < data['SPY_MACD_Signal']), 'Signal'] = -1  # Sell signal

    return data

# Apply the strategy to generate signals
merged_data = generate_signals(merged_data)

# Display sample data with signals to verify
merged_data[['Date', 'SPY_Close', 'SPY_RSI', 'SPY_MACD', 'SPY_MACD_Signal', 'Signal']].head(100)

,Date,SPY_Close,SPY_RSI,SPY_MACD,SPY_MACD_Signal,Signal
0,2016-01-04,174.043198,0.000000,NaN,NaN,0
1,2016-01-05,174.337585,0.000000,NaN,NaN,0
2,2016-01-06,172.138458,0.000000,NaN,NaN,0
3,2016-01-07,168.008591,0.000000,NaN,NaN,0
4,2016-01-08,166.164413,0.000000,NaN,NaN,0
...,...,...,...,...,...,...
95,2016-05-19,177.708313,43.924881,-0.167550,0.214988,0
96,2016-05-20,178.830933,48.594396,-0.174315,0.137127,0
97,2016-05-23,178.587296,47.666686,-0.197064,0.070289,0
98,2016-05-24,180.902191,56.219008,-0.027978,0.050636,0


In [6]:
# Remove rows with NaN values in essential columns (caused by initial indicator calculations)
cleaned_data = merged_data.dropna(subset=['SPY_MACD', 'SPY_MACD_Signal'])

# Define initial parameters for the PnL simulation
initial_capital = 1000000  # Starting capital in USD
position = 0  # Position in terms of SPY shares
cash = initial_capital  # Cash available
pnl = []  # Track PnL over time

# Simulate the PnL based on buy and sell signals
for index, row in cleaned_data.iterrows():
    if row['Signal'] == 1 and cash >= row['SPY_Close']:  # Buy signal
        # Buy as much SPY as possible with available cash
        position += cash / row['SPY_Close']
        cash = 0  # All cash invested
    elif row['Signal'] == -1 and position > 0:  # Sell signal
        # Sell all SPY holdings
        cash += position * row['SPY_Close']
        position = 0  # All shares sold
    # Track portfolio value: cash + current value of SPY holdings
    portfolio_value = cash + position * row['SPY_Close']
    pnl.append(portfolio_value)

# Add PnL results to the data for visualization
cleaned_data['PnL'] = pnl

# Display the final PnL value and the first few rows of the PnL data
final_pnl = pnl[-1]
cleaned_data[['Date', 'SPY_Close', 'Signal', 'PnL']].head(10), final_pnl

(          Date   SPY_Close  Signal        PnL
 33  2016-02-22  168.640594       0  1000000.0
 34  2016-02-23  166.510773       0  1000000.0
 35  2016-02-24  167.272629       0  1000000.0
 36  2016-02-25  169.298599       0  1000000.0
 37  2016-02-26  168.908966       0  1000000.0
 38  2016-02-29  167.584305       0  1000000.0
 39  2016-03-01  171.523727       0  1000000.0
 40  2016-03-02  172.294220       0  1000000.0
 41  2016-03-03  172.969620       0  1000000.0
 42  2016-03-04  173.532394       0  1000000.0,
 1000000.0)

In [7]:
# Redefine the buy and sell signal thresholds for a more lenient strategy
def generate_adjusted_signals(data):
    """
    Generate buy and sell signals with more lenient thresholds.
    Buy Signal: RSI < 40 and MACD > MACD_Signal
    Sell Signal: RSI > 60 and MACD < MACD_Signal
    """
    data['Signal'] = 0  # Reset to hold
    data.loc[(data['SPY_RSI'] < 40) & (data['SPY_MACD'] > data['SPY_MACD_Signal']), 'Signal'] = 1  # Buy signal
    data.loc[(data['SPY_RSI'] > 60) & (data['SPY_MACD'] < data['SPY_MACD_Signal']), 'Signal'] = -1  # Sell signal

    return data

# Apply the adjusted strategy to generate signals
adjusted_data = generate_adjusted_signals(cleaned_data)

# Reset PnL simulation with adjusted signals
position = 0  # Reset position
cash = initial_capital  # Reset cash to initial capital
adjusted_pnl = []  # Track adjusted PnL

# Simulate PnL based on adjusted buy/sell signals
for index, row in adjusted_data.iterrows():
    if row['Signal'] == 1 and cash >= row['SPY_Close']:  # Buy signal
        position += cash / row['SPY_Close']
        cash = 0  # All cash invested
    elif row['Signal'] == -1 and position > 0:  # Sell signal
        cash += position * row['SPY_Close']
        position = 0  # All shares sold
    # Track portfolio value
    portfolio_value = cash + position * row['SPY_Close']
    adjusted_pnl.append(portfolio_value)

# Add adjusted PnL results to the data for visualization
adjusted_data['Adjusted_PnL'] = adjusted_pnl

# Display the final PnL value and sample rows of the PnL data
final_adjusted_pnl = adjusted_pnl[-1]
adjusted_data[['Date', 'SPY_Close', 'Signal', 'Adjusted_PnL']].head(10), final_adjusted_pnl


(          Date   SPY_Close  Signal  Adjusted_PnL
 33  2016-02-22  168.640594       0     1000000.0
 34  2016-02-23  166.510773       0     1000000.0
 35  2016-02-24  167.272629       0     1000000.0
 36  2016-02-25  169.298599       0     1000000.0
 37  2016-02-26  168.908966       0     1000000.0
 38  2016-02-29  167.584305       0     1000000.0
 39  2016-03-01  171.523727       0     1000000.0
 40  2016-03-02  172.294220       0     1000000.0
 41  2016-03-03  172.969620       0     1000000.0
 42  2016-03-04  173.532394       0     1000000.0,
 1063502.5066111737)

In [8]:
adjusted_data['Signal'].value_counts()

Signal
 0    829
-1    142
 1      2
Name: count, dtype: int64

In [14]:
import numpy as np

# Calculate Total Return
initial_value = initial_capital
final_value = final_adjusted_pnl
total_return = (final_value - initial_value) / initial_value * 100

# Calculate Daily Returns from the Adjusted PnL values
adjusted_data['Daily_Returns'] = adjusted_data['Adjusted_PnL'].pct_change().fillna(0)

# Calculate Maximum Drawdown
cumulative_returns = (1 + adjusted_data['Daily_Returns']).cumprod()
rolling_max = cumulative_returns.cummax()
drawdown = cumulative_returns / rolling_max - 1
max_drawdown = drawdown.min() * 100  # Convert to percentage

# Calculate Sharpe Ratio
average_daily_return = adjusted_data['Daily_Returns'].mean()
std_dev_daily_return = adjusted_data['Daily_Returns'].std()
sharpe_ratio = (average_daily_return / std_dev_daily_return) * np.sqrt(252)  # Annualized Sharpe Ratio

# Calculate Sortino Ratio
downside_returns = adjusted_data.loc[adjusted_data['Daily_Returns'] < 0, 'Daily_Returns']
sortino_ratio = (average_daily_return / downside_returns.std()) * np.sqrt(252)  # Annualized Sortino Ratio

total_return, max_drawdown, sharpe_ratio, sortino_ratio


(6.350250661117373,
 np.float64(-11.207532488880256),
 np.float64(0.36280270601742476),
 np.float64(0.12146053285212845))

Here are the performance metrics for the adjusted strategy:

    Total Return: 6.35%
    Maximum Drawdown: -11.21% (indicating a significant peak-to-trough loss at some point)
    Sharpe Ratio: 0.36 (suggesting modest risk-adjusted returns)
    Sortino Ratio: 0.12 (indicating limited performance when adjusted for downside risk)

In [16]:
cleaned_data.columns

Index(['Date', 'SPY_Open', 'SPY_High', 'SPY_Low', 'SPY_Close', 'SPY_Volume',
       'SPY_Dividends', 'SPY_Stock Splits', 'SPY_Capital Gains', 'SPY_Ticker',
       'SPY_SMA_10', 'SPY_SMA_20', 'SPY_SMA_50', 'SPY_SMA_200',
       'SPY_daily_returns', 'SPY_weekly_returns', 'SPY_monthly_returns',
       'SPY_RSI', 'SPY_RSI2', 'SPY_MACD', 'SPY_MACD_Signal', 'SPY_Open_macd',
       'SPY_High_macd', 'SPY_Low_macd', 'SPY_Close_macd', 'SPY_Volume_macd',
       'SPY_Dividends_macd', 'SPY_Stock Splits_macd', 'SPY_Capital Gains_macd',
       'SPY_Ticker_macd', 'SPY_SMA_10_macd', 'SPY_SMA_20_macd',
       'SPY_SMA_50_macd', 'SPY_SMA_200_macd', 'SPY_daily_returns_macd',
       'SPY_weekly_returns_macd', 'SPY_monthly_returns_macd', 'SPY_RSI_macd',
       'SPY_RSI2_macd', 'SPY_MACD_macd', 'SPY_MACD_Signal_macd', 'Signal',
       'PnL', 'Adjusted_PnL', 'Daily_Returns'],
      dtype='object')

In [18]:
# Define optimized strategy with combined indicators
def generate_optimized_signals(data):
    """
    Generate buy/sell signals based on a combination of indicators:
    1. SMA crossover (short-term and long-term SMAs)
    2. RSI threshold (dynamic adjustment around 40 and 60)
    3. Volume confirmation (only buy when volume exceeds recent average)
    4. MACD signal confirmation
    """
    # Calculate short and long SMAs for crossover signals
    data['SMA_short'] = data['SPY_SMA_10']
    data['SMA_long'] = data['SPY_SMA_50']

    # Calculate 20-day average volume as a threshold for volume confirmation
    data['Volume_SMA20'] = data['SPY_Volume'].rolling(window=20).mean()

    # Reset signal column
    data['Signal'] = 0

    # Generate buy signal based on multiple conditions
    data.loc[
        (data['SMA_short'] > data['SMA_long']) &  # Short-term SMA above long-term
        (data['SPY_RSI'] < 45) &  # Lower RSI threshold for more frequent buy signals
        (data['SPY_MACD'] > data['SPY_MACD_Signal']) &  # MACD above signal line
        (data['SPY_Volume'] > data['Volume_SMA20']),  # Volume above 20-day average
        'Signal'
    ] = 1

    # Generate sell signal based on multiple conditions
    data.loc[
        (data['SMA_short'] < data['SMA_long']) &  # Short-term SMA below long-term
        (data['SPY_RSI'] > 55) &  # Higher RSI threshold for sell signals
        (data['SPY_MACD'] < data['SPY_MACD_Signal']),  # MACD below signal line
        'Signal'
    ] = -1

    return data

# Apply the optimized strategy to generate signals
optimized_data = generate_optimized_signals(cleaned_data)

# Reset PnL simulation with optimized signals
position = 0  # Reset position
cash = initial_capital  # Reset cash to initial capital
optimized_pnl = []  # Track optimized PnL

# Simulate PnL based on optimized buy/sell signals
for index, row in optimized_data.iterrows():
    if row['Signal'] == 1 and cash >= row['SPY_Close']:  # Buy signal
        position += cash / row['SPY_Close']
        cash = 0  # All cash invested
    elif row['Signal'] == -1 and position > 0:  # Sell signal
        cash += position * row['SPY_Close']
        position = 0  # All shares sold
    # Track portfolio value
    portfolio_value = cash + position * row['SPY_Close']
    optimized_pnl.append(portfolio_value)

# Add optimized PnL results to the data for visualization
optimized_data['Optimized_PnL'] = optimized_pnl

# Display the final PnL value and sample rows of the PnL data
final_optimized_pnl = optimized_pnl[-1]
optimized_data[['Date', 'SPY_Close', 'Signal', 'Optimized_PnL']].head(10), final_optimized_pnl


(          Date   SPY_Close  Signal  Optimized_PnL
 33  2016-02-22  168.640594       0      1000000.0
 34  2016-02-23  166.510773       0      1000000.0
 35  2016-02-24  167.272629       0      1000000.0
 36  2016-02-25  169.298599       0      1000000.0
 37  2016-02-26  168.908966       0      1000000.0
 38  2016-02-29  167.584305       0      1000000.0
 39  2016-03-01  171.523727       0      1000000.0
 40  2016-03-02  172.294220       0      1000000.0
 41  2016-03-03  172.969620       0      1000000.0
 42  2016-03-04  173.532394       0      1000000.0,
 1000000.0)

In [19]:
# Define a more lenient optimized strategy to increase signal frequency
def generate_more_signals(data):
    """
    Generate more frequent buy/sell signals by using:
    - Shorter SMA crossover period (SMA_10 vs. SMA_20)
    - Looser RSI thresholds (e.g., buy at RSI < 50, sell at RSI > 50)
    - Removal of volume-based confirmation
    """
    # Define shorter SMAs for faster crossover signals
    data['SMA_short'] = data['SPY_SMA_10']
    data['SMA_long'] = data['SPY_SMA_20']

    # Reset signal column
    data['Signal'] = 0

    # Generate buy signal with looser conditions
    data.loc[
        (data['SMA_short'] > data['SMA_long']) &  # Short-term SMA above long-term SMA
        (data['SPY_RSI'] < 50) &  # Looser buy condition for RSI
        (data['SPY_MACD'] > data['SPY_MACD_Signal']),  # MACD above signal line
        'Signal'
    ] = 1

    # Generate sell signal with looser conditions
    data.loc[
        (data['SMA_short'] < data['SMA_long']) &  # Short-term SMA below long-term SMA
        (data['SPY_RSI'] > 50) &  # Looser sell condition for RSI
        (data['SPY_MACD'] < data['SPY_MACD_Signal']),  # MACD below signal line
        'Signal'
    ] = -1

    return data

# Apply the modified strategy to generate more frequent signals
lenient_data = generate_more_signals(cleaned_data)

# Reset PnL simulation with lenient signals
position = 0  # Reset position
cash = initial_capital  # Reset cash to initial capital
lenient_pnl = []  # Track lenient PnL

# Simulate PnL based on lenient buy/sell signals
for index, row in lenient_data.iterrows():
    if row['Signal'] == 1 and cash >= row['SPY_Close']:  # Buy signal
        position += cash / row['SPY_Close']
        cash = 0  # All cash invested
    elif row['Signal'] == -1 and position > 0:  # Sell signal
        cash += position * row['SPY_Close']
        position = 0  # All shares sold
    # Track portfolio value
    portfolio_value = cash + position * row['SPY_Close']
    lenient_pnl.append(portfolio_value)

# Add lenient PnL results to the data for visualization
lenient_data['Lenient_PnL'] = lenient_pnl

# Display the final PnL value and sample rows of the PnL data
final_lenient_pnl = lenient_pnl[-1]
lenient_data[['Date', 'SPY_Close', 'Signal', 'Lenient_PnL']].head(10), final_lenient_pnl


(          Date   SPY_Close  Signal  Lenient_PnL
 33  2016-02-22  168.640594       0    1000000.0
 34  2016-02-23  166.510773       0    1000000.0
 35  2016-02-24  167.272629       0    1000000.0
 36  2016-02-25  169.298599       0    1000000.0
 37  2016-02-26  168.908966       0    1000000.0
 38  2016-02-29  167.584305       0    1000000.0
 39  2016-03-01  171.523727       0    1000000.0
 40  2016-03-02  172.294220       0    1000000.0
 41  2016-03-03  172.969620       0    1000000.0
 42  2016-03-04  173.532394       0    1000000.0,
 1000000.0)

In [20]:
# Define a strategy using three moving averages for buy/sell signals
def generate_three_sma_signals(data):
    """
    Generate buy/sell signals using a three moving average crossover:
    - Buy when the short-term SMA crosses above both medium-term and long-term SMAs
    - Sell when the short-term SMA crosses below both medium-term and long-term SMAs
    """
    # Define short, medium, and long SMAs
    data['SMA_short'] = data['SPY_SMA_5'] if 'SPY_SMA_5' in data else data['SPY_SMA_10']  # fallback to SMA_10 if SMA_5 not available
    data['SMA_medium'] = data['SPY_SMA_10']
    data['SMA_long'] = data['SPY_SMA_20']

    # Reset signal column
    data['Signal'] = 0

    # Generate buy signal: short-term above both medium and long-term
    data.loc[
        (data['SMA_short'] > data['SMA_medium']) &
        (data['SMA_medium'] > data['SMA_long']) &
        (data['SPY_MACD'] > data['SPY_MACD_Signal']),  # optional MACD confirmation
        'Signal'
    ] = 1

    # Generate sell signal: short-term below both medium and long-term
    data.loc[
        (data['SMA_short'] < data['SMA_medium']) &
        (data['SMA_medium'] < data['SMA_long']) &
        (data['SPY_MACD'] < data['SPY_MACD_Signal']),  # optional MACD confirmation
        'Signal'
    ] = -1

    return data

# Apply the three-SMA strategy to generate signals
three_sma_data = generate_three_sma_signals(cleaned_data)

# Reset PnL simulation with three-SMA signals
position = 0  # Reset position
cash = initial_capital  # Reset cash to initial capital
three_sma_pnl = []  # Track three-SMA PnL

# Simulate PnL based on three-SMA buy/sell signals
for index, row in three_sma_data.iterrows():
    if row['Signal'] == 1 and cash >= row['SPY_Close']:  # Buy signal
        position += cash / row['SPY_Close']
        cash = 0  # All cash invested
    elif row['Signal'] == -1 and position > 0:  # Sell signal
        cash += position * row['SPY_Close']
        position = 0  # All shares sold
    # Track portfolio value
    portfolio_value = cash + position * row['SPY_Close']
    three_sma_pnl.append(portfolio_value)

# Add three-SMA PnL results to the data for visualization
three_sma_data['Three_SMA_PnL'] = three_sma_pnl

# Display the final PnL value and sample rows of the PnL data
final_three_sma_pnl = three_sma_pnl[-1]
three_sma_data[['Date', 'SPY_Close', 'Signal', 'Three_SMA_PnL']].head(10), final_three_sma_pnl


(          Date   SPY_Close  Signal  Three_SMA_PnL
 33  2016-02-22  168.640594       0      1000000.0
 34  2016-02-23  166.510773       0      1000000.0
 35  2016-02-24  167.272629       0      1000000.0
 36  2016-02-25  169.298599       0      1000000.0
 37  2016-02-26  168.908966       0      1000000.0
 38  2016-02-29  167.584305       0      1000000.0
 39  2016-03-01  171.523727       0      1000000.0
 40  2016-03-02  172.294220       0      1000000.0
 41  2016-03-03  172.969620       0      1000000.0
 42  2016-03-04  173.532394       0      1000000.0,
 1000000.0)

In [21]:
# Adjusted strategy using shorter SMAs for quicker signals
def generate_short_sma_signals(data):
    """
    Generate buy/sell signals using short SMAs:
    - Short-term: 3-day SMA
    - Medium-term: 7-day SMA
    - Long-term: 15-day SMA
    Buy when the 3-day SMA crosses above both 7- and 15-day SMAs.
    Sell when the 3-day SMA crosses below both 7- and 15-day SMAs.
    """
    # Set short, medium, and long SMAs with shorter periods
    data['SMA_short'] = data['SPY_Close'].rolling(window=3).mean()
    data['SMA_medium'] = data['SPY_Close'].rolling(window=7).mean()
    data['SMA_long'] = data['SPY_Close'].rolling(window=15).mean()

    # Reset signal column
    data['Signal'] = 0

    # Generate buy signal: short-term above both medium and long-term
    data.loc[
        (data['SMA_short'] > data['SMA_medium']) &
        (data['SMA_medium'] > data['SMA_long']) &
        (data['SPY_MACD'] > data['SPY_MACD_Signal']),  # MACD confirmation
        'Signal'
    ] = 1

    # Generate sell signal: short-term below both medium and long-term
    data.loc[
        (data['SMA_short'] < data['SMA_medium']) &
        (data['SMA_medium'] < data['SMA_long']) &
        (data['SPY_MACD'] < data['SPY_MACD_Signal']),  # MACD confirmation
        'Signal'
    ] = -1

    return data

# Apply the short SMA strategy to generate signals
short_sma_data = generate_short_sma_signals(cleaned_data)

# Reset PnL simulation with short SMA signals
position = 0  # Reset position
cash = initial_capital  # Reset cash to initial capital
short_sma_pnl = []  # Track short SMA PnL

# Simulate PnL based on short SMA buy/sell signals
for index, row in short_sma_data.iterrows():
    if row['Signal'] == 1 and cash >= row['SPY_Close']:  # Buy signal
        position += cash / row['SPY_Close']
        cash = 0  # All cash invested
    elif row['Signal'] == -1 and position > 0:  # Sell signal
        cash += position * row['SPY_Close']
        position = 0  # All shares sold
    # Track portfolio value
    portfolio_value = cash + position * row['SPY_Close']
    short_sma_pnl.append(portfolio_value)

# Add short SMA PnL results to the data for visualization
short_sma_data['Short_SMA_PnL'] = short_sma_pnl

# Display the final PnL value and sample rows of the PnL data
final_short_sma_pnl = short_sma_pnl[-1]
short_sma_data[['Date', 'SPY_Close', 'Signal', 'Short_SMA_PnL']].head(10), final_short_sma_pnl


(          Date   SPY_Close  Signal  Short_SMA_PnL
 33  2016-02-22  168.640594       0      1000000.0
 34  2016-02-23  166.510773       0      1000000.0
 35  2016-02-24  167.272629       0      1000000.0
 36  2016-02-25  169.298599       0      1000000.0
 37  2016-02-26  168.908966       0      1000000.0
 38  2016-02-29  167.584305       0      1000000.0
 39  2016-03-01  171.523727       0      1000000.0
 40  2016-03-02  172.294220       0      1000000.0
 41  2016-03-03  172.969620       0      1000000.0
 42  2016-03-04  173.532394       0      1000000.0,
 1277162.1332324825)

In [22]:
# Calculate risk metrics for the three-SMA strategy with adjusted short SMA lengths

# Total Return
total_return_short_sma = (final_short_sma_pnl - initial_value) / initial_value * 100

# Calculate daily returns based on the Short_SMA_PnL
short_sma_data['Daily_Returns'] = short_sma_data['Short_SMA_PnL'].pct_change().fillna(0)

# Maximum Drawdown
cumulative_returns_short = (1 + short_sma_data['Daily_Returns']).cumprod()
rolling_max_short = cumulative_returns_short.cummax()
drawdown_short = cumulative_returns_short / rolling_max_short - 1
max_drawdown_short = drawdown_short.min() * 100  # Convert to percentage

# Sharpe Ratio
average_daily_return_short = short_sma_data['Daily_Returns'].mean()
std_dev_daily_return_short = short_sma_data['Daily_Returns'].std()
sharpe_ratio_short = (average_daily_return_short / std_dev_daily_return_short) * np.sqrt(252)  # Annualized Sharpe Ratio

# Sortino Ratio
downside_returns_short = short_sma_data.loc[short_sma_data['Daily_Returns'] < 0, 'Daily_Returns']
sortino_ratio_short = (average_daily_return_short / downside_returns_short.std()) * np.sqrt(252)  # Annualized Sortino Ratio

total_return_short_sma, max_drawdown_short, sharpe_ratio_short, sortino_ratio_short


(27.716213323248244,
 np.float64(-19.570354742287144),
 np.float64(0.8748422331079568),
 np.float64(0.812953813038033))

Total Return: 27.72%

    Maximum Drawdown: -19.57% (showing a significant peak-to-trough decline)
    Sharpe Ratio: 0.87 (indicating moderate risk-adjusted returns)
    Sortino Ratio: 0.81 (focused on downside risk)